In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train=pd.read_csv("C:/Users/Pranita/DL_P/Notebooks/DATA/Google_Stock_Price_Train.csv")

In [3]:
train.head()

,Date,Open,High,Low,Close,Volume
0,1/3/2012,325.25,332.83,324.97,663.59,"7,380,500"
1,1/4/2012,331.27,333.87,329.08,666.45,"5,749,400"
2,1/5/2012,329.83,330.75,326.89,657.21,"6,590,300"
3,1/6/2012,328.34,328.77,323.68,648.24,"5,405,900"
4,1/9/2012,322.04,322.29,309.46,620.76,"11,688,800"


In [4]:
train.shape

(1258, 6)

In [5]:
A=train.to_numpy()[:,1:2]

In [6]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
A_scaled = sc.fit_transform(A)

In [7]:
xtrain=[]
ytrain=[]
for i in range(60,1258):
    xtrain.append(A_scaled[i-60:i,0])
    ytrain.append(A_scaled[i,0])

In [8]:
xtrain=np.array(xtrain)
ytrain=np.array(ytrain)

In [9]:
A_scaled.dtype

dtype('float64')

In [10]:
xtrain = np.reshape(xtrain, (xtrain.shape[0], xtrain.shape[1], 1))

In [11]:
import tensorflow as tf

In [12]:
model=tf.keras.Sequential()

model.add(tf.keras.layers.LSTM(units=50,return_sequences=True,input_shape=(xtrain.shape[1],1)))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.LSTM(units=50,return_sequences=True))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.LSTM(units=50,return_sequences=True))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(units=1))

model.compile(optimizer="adam",loss="mean_squared_error",metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 60, 50)            10400     
_________________________________________________________________
dropout (Dropout)            (None, 60, 50)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 60, 50)            20200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 50)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 60, 50)            20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 60, 50)            0         
_________________________________________________________________
dense (Dense)                (None, 60, 1)             5

In [13]:
history=model.fit(xtrain,ytrain,epochs=10,batch_size=32)

Epoch 1/10
38/38 [==============================] - 4s 102ms/step - loss: 0.0644 - accuracy: 0.0015
Epoch 2/10
38/38 [==============================] - 4s 99ms/step - loss: 0.0244 - accuracy: 0.0016
Epoch 3/10
38/38 [==============================] - 4s 100ms/step - loss: 0.0195 - accuracy: 0.0016
Epoch 4/10
38/38 [==============================] - 4s 101ms/step - loss: 0.0166 - accuracy: 0.0016
Epoch 5/10
38/38 [==============================] - 4s 102ms/step - loss: 0.0149 - accuracy: 0.0016
Epoch 6/10
38/38 [==============================] - 4s 101ms/step - loss: 0.0139 - accuracy: 0.0016
Epoch 7/10
38/38 [==============================] - 4s 102ms/step - loss: 0.0126 - accuracy: 0.0016
Epoch 8/10
38/38 [==============================] - 4s 102ms/step - loss: 0.0119 - accuracy: 0.0016
Epoch 9/10
38/38 [==============================] - 4s 104ms/step - loss: 0.0112 - accuracy: 0.0016
Epoch 10/10
38/38 [==============================] - 4s 102ms/step - loss: 0.0106 - accuracy: 0.0016


In [14]:
B=pd.read_csv("C:/Users/Pranita/DL_P/Notebooks/DATA/Google_Stock_Price_Test.csv")
B.head()

,Date,Open,High,Low,Close,Volume
0,1/3/2017,778.81,789.63,775.80,786.14,"1,657,300"
1,1/4/2017,788.36,791.34,783.16,786.90,"1,073,000"
2,1/5/2017,786.08,794.48,785.02,794.02,"1,335,200"
3,1/6/2017,795.26,807.90,792.20,806.15,"1,640,200"
4,1/9/2017,806.40,809.97,802.83,806.65,"1,272,400"


In [15]:
real_stock_price=B.to_numpy()[:,1:2]

In [16]:
tot = pd.concat((train['Open'], B['Open']), axis = 0)
C = tot[len(tot) - len(B) - 60:].values
C = C.reshape(-1,1)
C = sc.transform(C)

In [17]:
xtest = []
ytest=[]
for i in range(60, 80):
    xtest.append(C[i-60:i, 0])
    ytest.append(C[i,0])
xtest,ytest = np.array(xtest),np.array(ytest)
xtest = np.reshape(xtest, (xtest.shape[0], xtest.shape[1], 1))
pred = model.predict(xtest)


In [19]:
pred=pred.reshape(20,60)
pred = sc.inverse_transform(pred)